In [1]:
import pycaret as pc
import pandas as pd

In [30]:
sell_data = pd.read_excel('./HistoryOfSell_Rev02.xlsx', sheet_name='Sell')

sell_data.head()

,Date,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24
0,2014-03-01,2,2,0,1,1,0,2,0,1,...,0,0,1,0,1,0,1,1,0,0
1,2014-04-01,6,4,0,2,4,0,1,1,0,...,0,1,0,2,6,3,5,5,2,2
2,2014-05-01,9,3,0,3,6,0,0,1,0,...,0,0,2,0,4,1,3,1,0,0
3,2014-06-01,8,3,2,6,6,0,2,2,4,...,0,0,2,0,6,2,6,3,0,0
4,2014-07-01,11,4,0,6,4,0,2,3,3,...,0,0,2,0,9,0,6,5,0,0


In [31]:
sell_data['A1_Mean'] = sell_data['A1'].rolling(12).mean()

import plotly.express as px

fig = px.line(sell_data, x='Date', y=['A1', 'A1_Mean'], template='plotly_dark')
fig.show()

In [32]:
import numpy as np

# extract month and year from dates
sell_data['Month'] = [i.month for i in sell_data['Date']]
sell_data['Year'] = [i.year for i in sell_data['Date']]

# create a sequence of numbers
sell_data['Series'] = np.arange(1,len(sell_data)+1)

# drop unnecessary columns and re-arrange
sell_data.drop(['Date', 'A1_Mean'], axis=1, inplace=True)
sell_data = sell_data[['Series', 'Year', 'Month', 'A1']] 

# check the head of the dataset
sell_data.head()

,Series,Year,Month,A1
0,1,2014,3,2
1,2,2014,4,6
2,3,2014,5,9
3,4,2014,6,8
4,5,2014,7,11


In [33]:
# split data into train-test set
train = sell_data[sell_data['Year'] < 2022]
test = sell_data[sell_data['Year'] >= 2022]

# check shape
train.shape, test.shape


((94, 4), (17, 4))

In [34]:
# import the regression module
from pycaret.regression import *

# initialize setup
s = setup(data = train, test_data = test, target = 'A1', fold_strategy = 'timeseries', numeric_features = ['Year', 'Series'], fold = 10, transform_target = True, session_id = 123)

,Description,Value
0,Session id,123
1,Target,A1
2,Target type,Regression
3,Original data shape,"(111, 4)"
4,Transformed data shape,"(111, 4)"
5,Transformed train set shape,"(94, 4)"
6,Transformed test set shape,"(17, 4)"
7,Numeric features,2
8,Preprocess,1
9,Imputation type,simple


In [35]:
best = compare_models(sort = 'MSE')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:03:55
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,3.6698,24.1117,4.4322,-0.0086,0.4543,0.4498,0.2170
dummy,Dummy Regressor,4.0818,26.9577,4.8378,-0.2913,0.5132,0.5566,0.0720
llar,Lasso Least Angle Regression,4.0818,26.9577,4.8378,-0.2913,0.5132,0.5566,0.0580
rf,Random Forest Regressor,3.7501,26.9599,4.6546,-0.1390,0.4872,0.4774,0.2290
knn,K Neighbors Regressor,3.8168,27.4388,4.7833,-0.1875,0.4916,0.4719,0.0610
gbr,Gradient Boosting Regressor,4.0155,27.5015,4.8422,-0.3330,0.5138,0.5476,0.1110
ada,AdaBoost Regressor,3.7495,27.6648,4.8289,-0.4137,0.4812,0.4628,0.1270
lasso,Lasso Regression,4.2448,29.1467,4.9995,-0.3728,0.5288,0.5552,0.0590
lightgbm,Light Gradient Boosting Machine,4.1199,29.1716,5.0111,-0.3748,0.5211,0.5245,0.0830
br,Bayesian Ridge,4.5040,32.6903,5.3437,-0.7496,0.5452,0.5762,0.0640


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [36]:
prediction_holdout = predict_model(best);

In [25]:
predictions

,Series,Year,Month,A1,prediction_label,Date
0,1,2014,3,1.256027,3.020646,2014-03-01
1,2,2014,4,2.476401,6.431157,2014-04-01
2,3,2014,5,3.068632,8.596421,2014-05-01
3,4,2014,6,2.888415,8.631779,2014-06-01
4,5,2014,7,3.391394,10.114211,2014-07-01
...,...,...,...,...,...,...
106,107,2023,1,3.537588,12.814824,2023-01-01
107,108,2023,2,5.168150,14.086215,2023-02-01
108,109,2023,3,2.476401,4.859511,2023-03-01
109,110,2023,4,3.537588,4.662249,2023-04-01


In [37]:
# generate predictions on the original dataset**
predictions = predict_model(best, data=sell_data)

# add a date column in the dataset**
predictions['Date'] = pd.date_range(start='2014-03-01', end = '2023-05-01', freq = 'MS')

# line plot**
fig = px.line(predictions, x='Date', y=["A1", "prediction_label"], template = 'plotly_dark')

# add a vertical rectange for test-set separation**
fig.add_vrect(x0="2022-07-01", x1="2023-06-01", fillcolor="grey", opacity=0.25, line_width=0)

fig.show()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.8792,8.2289,2.8686,0.7431,0.2243,0.0700


In [38]:
final_best = finalize_model(best)

In [48]:
future_dates = pd.date_range(start = '2023-06-01', end = '2024-05-01', freq = 'MS')

future_df = pd.DataFrame()

future_df['Month'] = [i.month for i in future_dates]
future_df['Year'] = [i.year for i in future_dates]    
future_df['Series'] = np.arange(112,(112+len(future_dates)))

future_df.head()

,Month,Year,Series
0,6,2023,112
1,7,2023,113
2,8,2023,114
3,9,2023,115
4,10,2023,116


In [49]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future.head()

,Month,Year,Series,prediction_label
0,6,2023,112,11.203337
1,7,2023,113,11.355895
2,8,2023,114,10.739576
3,9,2023,115,10.918457
4,10,2023,116,13.118724


In [50]:
concat_df = pd.concat([sell_data, predictions_future], axis=0)
concat_df_i = pd.date_range(start='2014-03-01', end = '2024-05-01', freq = 'MS')
concat_df.set_index(concat_df_i, inplace=True)

fig = px.line(concat_df, x=concat_df.index, y=["A1", "prediction_label"], template = 'plotly_dark')
fig.show()